In [1]:
import pandas as pd
import nltk.corpus
import string
import re
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import LancasterStemmer
import nltk
from nltk.corpus import stopwords
from sklearn.multiclass import OneVsRestClassifier

In [2]:
file = pd.read_csv("train_F3WbcTw.csv")
file = file.drop(["unique_hash","drug"],axis=1)
file2 = pd.read_csv("test_tOlRoBf.csv")
file2 = file2.drop(["unique_hash","drug"],axis=1)

In [3]:
file["clean text"] = file["text"].apply( lambda x : x.lower())
file["clean text"] = file["clean text"].str.replace("[^a-zA-Z#]"," ")
file["clean text"] = file["clean text"].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
file["clean text"] = file["clean text"].apply( lambda x : x.split(" "))
stemmer = PorterStemmer()
file["clean text"] = file["clean text"].apply(lambda x:[stemmer.stem(i) for i in x] )
file["clean text"] = file["clean text"].apply(lambda x:' '.join(x) )
file = file.drop(["text"], axis = 1)
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=None, stop_words='english', ngram_range=(1, 2))
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(file["clean text"])

file2["clean text"] = file2["text"].apply( lambda x : x.lower())
file2["clean text"] = file2["clean text"].str.replace("[^a-zA-Z#]"," ")
file2["clean text"] = file2["clean text"].apply(lambda x: " ".join([w for w in x.split() if len(w)>3]))
file2["clean text"] = file2["clean text"].apply( lambda x : x.split(" "))

file2["clean text"] = file2["clean text"].apply(lambda x:[stemmer.stem(i) for i in x] )
file2["clean text"] = file2["clean text"].apply(lambda x:' '.join(x) )
file2 = file2.drop(["text"], axis = 1)


# bag-of-words feature matrix
bow2 = bow_vectorizer.transform(file2["clean text"])

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np


y = file["sentiment"]

#splitting data into training and validation set
xtrain, xtest, ytrain, ytest = train_test_split(bow,y,random_state=42, test_size=0.3)
lreg = OneVsRestClassifier(LogisticRegression())


lreg.fit(xtrain, ytrain) # training the model
predict = lreg.predict(xtest)
from sklearn import metrics
#metrics.accuracy_score(predict,ytest)
print("macro f1 score:",metrics.f1_score(predict,ytest,average = "macro"))
#print("micro f1 score:",metrics.f1_score(predict,ytest,average = "micro"))

macro f1 score: 0.4506488853503184


In [82]:
print(predict)
len(predict)

[2 2 2 ... 0 2 2]


2924

In [46]:
predict

array([2, 2, 2, ..., 0, 2, 2], dtype=int64)

In [87]:
submission = pd.DataFrame({'unique_hash':file3['unique_hash'],'sentiment':predict})

In [89]:
len(submission)

2924

In [91]:
submission.to_csv("predict.csv",index=False)